# Q-Learning 基础与实现

## 学习目标

通过本教程，你将掌握：
- 时序差分学习 (TD Learning) 的核心思想
- Q-Learning 算法的数学原理与推导
- 表格型 Q-Learning 的工程实现
- 探索与利用的平衡策略

## 前置知识

- 马尔可夫决策过程 (MDP) 基础
- Python 与 NumPy 基础
- 概率论基础

---

## 第一部分：从动态规划到无模型学习

### 1.1 动态规划的局限性

动态规划 (DP) 方法需要：
1. **完整的环境模型**：$P(s'|s,a)$ 和 $R(s,a,s')$
2. **遍历所有状态和动作**

**现实挑战**:
- 转移概率通常未知
- 状态空间可能极其巨大（围棋约有 $10^{170}$ 种状态）

### 1.2 无模型强化学习

**核心思想**: 通过与环境**交互采样**学习最优策略，无需事先知道环境模型。

```
┌─────────────────────────────────────────────────────────┐
│                强化学习方法分类                          │
├─────────────────────────────────────────────────────────┤
│  基于模型            │        无模型 (Model-Free)        │
│  (Model-Based)       │  基于价值      │    基于策略      │
│  • 动态规划          │  • Q-Learning  │    • REINFORCE   │
│  • 模型预测控制      │  • SARSA       │    • PPO         │
│  • Dyna-Q           │  • DQN         │    • A3C         │
└─────────────────────────────────────────────────────────┘
```

---

## 第二部分：时序差分学习 (TD Learning)

### 2.1 蒙特卡洛 vs 时序差分

**蒙特卡洛方法** (Monte Carlo):

$$V(S_t) \leftarrow V(S_t) + \alpha \left[ G_t - V(S_t) \right]$$

- 需要等待**回合结束**才能更新
- $G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + ...$

**时序差分方法** (TD):

$$V(S_t) \leftarrow V(S_t) + \alpha \left[ R_{t+1} + \gamma V(S_{t+1}) - V(S_t) \right]$$

- **每一步**都可以更新
- 使用**自举** (Bootstrapping)：用估计值更新估计值

### 2.2 TD 误差

**TD 目标**: $\text{TD Target} = R_{t+1} + \gamma V(S_{t+1})$

**TD 误差**: $\delta_t = R_{t+1} + \gamma V(S_{t+1}) - V(S_t)$

| $\delta_t$ | 含义 | 操作 |
|------------|------|------|
| $> 0$ | 实际比预期好 | 增大 $V(S_t)$ |
| $< 0$ | 实际比预期差 | 减小 $V(S_t)$ |
| $= 0$ | 预测准确 | 已收敛 |

---

## 第三部分：Q-Learning 算法

### 3.1 算法原理

Q-Learning 是**离策略** (Off-Policy) TD 控制算法。

**更新公式**:

$$Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha \left[ R_{t+1} + \gamma \max_{a} Q(S_{t+1}, a) - Q(S_t, A_t) \right]$$

**关键特性**:
- 使用 $\max$ 操作选择下一状态的最大 Q 值
- 与当前实际采取的行为策略无关
- 在一定条件下保证收敛到最优 Q 函数

### 3.2 算法伪代码

```
算法: Q-Learning

1. 初始化 Q(s, a) = 0
2. 对于每个回合:
   a. 初始化状态 S
   b. 重复:
      i.   使用 ε-greedy 从 Q 选择动作 A
      ii.  执行 A，观察 R, S'
      iii. Q(S, A) ← Q(S, A) + α[R + γ max_a Q(S', a) - Q(S, A)]
      iv.  S ← S'
   c. 直到终止
```

---

## 第四部分：代码实现

In [ ]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from typing import Tuple, List, Dict, Any

# 设置随机种子
np.random.seed(42)

# 可视化配置
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

print("环境配置完成")

### 4.1 悬崖行走环境

悬崖行走 (Cliff Walking) 是经典的强化学习测试环境：

```
┌─────────────────────────────────────────────┐
│ .  .  .  .  .  .  .  .  .  .  .  .  │  row 0
│ .  .  .  .  .  .  .  .  .  .  .  .  │  row 1
│ .  .  .  .  .  .  .  .  .  .  .  .  │  row 2
│ S  C  C  C  C  C  C  C  C  C  C  G  │  row 3
└─────────────────────────────────────────────┘

S: 起点    G: 目标    C: 悬崖
动作: 上(0)、右(1)、下(2)、左(3)
奖励: 每步 -1，掉入悬崖 -100
```

In [ ]:
class CliffWalkingEnv:
    """悬崖行走环境"""
    
    ACTIONS = {0: (-1, 0), 1: (0, 1), 2: (1, 0), 3: (0, -1)}
    ACTION_NAMES = ['上', '右', '下', '左']
    
    def __init__(self, height: int = 4, width: int = 12):
        self.height = height
        self.width = width
        self.start = (height - 1, 0)
        self.goal = (height - 1, width - 1)
        self.cliff = [(height - 1, j) for j in range(1, width - 1)]
        self.state = self.start
        self.n_actions = 4

In [ ]:
    def reset(self) -> Tuple[int, int]:
        """重置环境"""
        self.state = self.start
        return self.state

In [ ]:
    def step(self, action: int) -> Tuple[Tuple[int, int], float, bool]:
        """执行动作"""
        di, dj = self.ACTIONS[action]
        new_i = int(np.clip(self.state[0] + di, 0, self.height - 1))
        new_j = int(np.clip(self.state[1] + dj, 0, self.width - 1))
        next_state = (new_i, new_j)
        
        # 悬崖检查
        if next_state in self.cliff:
            self.state = self.start
            return self.state, -100.0, False
        
        self.state = next_state
        
        # 目标检查
        if self.state == self.goal:
            return self.state, 0.0, True
        
        return self.state, -1.0, False

In [ ]:
    def render(self, path=None):
        """可视化环境"""
        grid = [['.' for _ in range(self.width)] for _ in range(self.height)]
        for pos in self.cliff:
            grid[pos[0]][pos[1]] = 'C'
        grid[self.start[0]][self.start[1]] = 'S'
        grid[self.goal[0]][self.goal[1]] = 'G'
        if path:
            for pos in path[1:-1]:
                if pos not in self.cliff and pos != self.start and pos != self.goal:
                    grid[pos[0]][pos[1]] = '*'
        print("┌" + "─" * (self.width * 2 + 1) + "┐")
        for row in grid:
            print("│ " + " ".join(row) + " │")
        print("└" + "─" * (self.width * 2 + 1) + "┘")

In [ ]:
# 将方法添加到类中并测试
CliffWalkingEnv.reset = reset
CliffWalkingEnv.step = step
CliffWalkingEnv.render = render

env = CliffWalkingEnv()
print("悬崖行走环境:")
env.render()
print(f"\n起点: {env.start}")
print(f"终点: {env.goal}")

### 4.2 Q-Learning 智能体

In [ ]:
class QLearningAgent:
    """表格型 Q-Learning 智能体"""
    
    def __init__(
        self,
        n_actions: int,
        learning_rate: float = 0.1,
        discount_factor: float = 0.99,
        epsilon: float = 1.0,
        epsilon_decay: float = 0.995,
        epsilon_min: float = 0.01
    ):
        self.n_actions = n_actions
        self.lr = learning_rate
        self.gamma = discount_factor
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.q_table = defaultdict(lambda: np.zeros(n_actions))

In [ ]:
    def get_action(self, state, training: bool = True) -> int:
        """
        ε-greedy 动作选择
        
        π(a|s) = ε/|A| + (1-ε)·𝟙[a = argmax Q]
        """
        if training and np.random.random() < self.epsilon:
            return np.random.randint(self.n_actions)
        
        q_values = self.q_table[state]
        max_q = np.max(q_values)
        max_actions = np.where(np.isclose(q_values, max_q))[0]
        return np.random.choice(max_actions)

In [ ]:
    def update(self, state, action, reward, next_state, done) -> float:
        """
        Q-Learning 更新规则
        
        Q(s,a) ← Q(s,a) + α[r + γ max_a Q(s',a) - Q(s,a)]
        """
        current_q = self.q_table[state][action]
        
        if done:
            target = reward
        else:
            target = reward + self.gamma * np.max(self.q_table[next_state])
        
        td_error = target - current_q
        self.q_table[state][action] += self.lr * td_error
        
        return td_error

In [ ]:
    def decay_epsilon(self):
        """衰减探索率"""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

In [ ]:
# 组装完整类
QLearningAgent.get_action = get_action
QLearningAgent.update = update
QLearningAgent.decay_epsilon = decay_epsilon

# 创建智能体
agent = QLearningAgent(
    n_actions=4,
    learning_rate=0.5,
    epsilon=0.1,
    epsilon_decay=1.0,
    epsilon_min=0.1
)

print("Q-Learning 智能体配置:")
print(f"  学习率 α: {agent.lr}")
print(f"  折扣因子 γ: {agent.gamma}")
print(f"  探索率 ε: {agent.epsilon}")

### 4.3 训练循环

In [ ]:
def train_q_learning(env, agent, episodes=500, max_steps=200, verbose=True):
    """训练 Q-Learning 智能体"""
    history = {'rewards': [], 'steps': [], 'epsilon': []}
    
    for episode in range(episodes):
        state = env.reset()
        total_reward = 0.0
        steps = 0
        
        for _ in range(max_steps):
            action = agent.get_action(state, training=True)
            next_state, reward, done = env.step(action)
            agent.update(state, action, reward, next_state, done)
            
            state = next_state
            total_reward += reward
            steps += 1
            
            if done:
                break
        
        agent.decay_epsilon()
        history['rewards'].append(total_reward)
        history['steps'].append(steps)
        history['epsilon'].append(agent.epsilon)
        
        if verbose and (episode + 1) % 100 == 0:
            avg_reward = np.mean(history['rewards'][-100:])
            print(f"Episode {episode+1:4d} | Avg Reward: {avg_reward:8.2f} | ε: {agent.epsilon:.4f}")
    
    return history

In [ ]:
# 训练智能体
print("开始训练 Q-Learning...\n")
env = CliffWalkingEnv()
agent = QLearningAgent(n_actions=4, learning_rate=0.5, epsilon=0.1, epsilon_decay=1.0, epsilon_min=0.1)
history = train_q_learning(env, agent, episodes=500)
print(f"\n训练完成！最后100回合平均奖励: {np.mean(history['rewards'][-100:]):.2f}")

### 4.4 可视化学习过程

In [ ]:
def plot_training(history, window=10):
    """绘制训练曲线"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 奖励曲线
    rewards = history['rewards']
    smoothed = np.convolve(rewards, np.ones(window)/window, mode='valid')
    
    axes[0].plot(rewards, alpha=0.3, color='blue', label='原始')
    axes[0].plot(range(window-1, len(rewards)), smoothed, color='blue', linewidth=2, label='平滑')
    axes[0].set_xlabel('Episode')
    axes[0].set_ylabel('Total Reward')
    axes[0].set_title('学习曲线')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # 步数曲线
    steps = history['steps']
    smoothed_steps = np.convolve(steps, np.ones(window)/window, mode='valid')
    
    axes[1].plot(steps, alpha=0.3, color='green', label='原始')
    axes[1].plot(range(window-1, len(steps)), smoothed_steps, color='green', linewidth=2, label='平滑')
    axes[1].set_xlabel('Episode')
    axes[1].set_ylabel('Steps')
    axes[1].set_title('回合步数')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
plot_training(history)

### 4.5 提取学到的策略

In [ ]:
def extract_path(agent, env, max_steps=50):
    """提取贪心策略路径"""
    state = env.reset()
    path = [state]
    
    for _ in range(max_steps):
        action = agent.get_action(state, training=False)
        next_state, _, done = env.step(action)
        path.append(next_state)
        state = next_state
        if done:
            break
    
    return path

In [ ]:
def visualize_policy(agent, env):
    """可视化策略"""
    arrow_map = {0: '↑', 1: '→', 2: '↓', 3: '←'}
    
    print("学到的策略:")
    print("┌" + "───" * env.width + "┐")
    
    for i in range(env.height):
        row = "│"
        for j in range(env.width):
            state = (i, j)
            if state == env.start:
                row += " S "
            elif state == env.goal:
                row += " G "
            elif state in env.cliff:
                row += " C "
            elif state in agent.q_table:
                best_action = np.argmax(agent.q_table[state])
                row += f" {arrow_map[best_action]} "
            else:
                row += " . "
        print(row + "│")
    
    print("└" + "───" * env.width + "┘")

In [ ]:
# 可视化策略
visualize_policy(agent, env)

# 显示路径
print("\n学到的路径:")
path = extract_path(agent, env)
env.reset()
env.render(path)
print(f"路径长度: {len(path) - 1} 步")

### 4.6 Q 值表可视化

In [ ]:
def visualize_q_table(agent, env):
    """可视化 Q 表"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 价值函数
    v_table = np.zeros((env.height, env.width))
    for i in range(env.height):
        for j in range(env.width):
            state = (i, j)
            if state in agent.q_table:
                v_table[i, j] = np.max(agent.q_table[state])
    
    im = axes[0].imshow(v_table, cmap='RdYlGn', aspect='auto')
    axes[0].set_title('V(s) = max_a Q(s,a)')
    plt.colorbar(im, ax=axes[0])
    
    for pos in env.cliff:
        axes[0].add_patch(plt.Rectangle((pos[1]-0.5, pos[0]-0.5), 1, 1, fill=True, color='black', alpha=0.5))
    
    # Q 值分布
    q_values = []
    for q_array in agent.q_table.values():
        q_values.extend(q_array.tolist())
    
    axes[1].hist(q_values, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
    axes[1].set_xlabel('Q Value')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Q 值分布')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
visualize_q_table(agent, env)

---

## 第五部分：探索策略

### 5.1 探索与利用困境

| 策略 | 描述 | 风险 |
|------|------|------|
| **利用** | 选择当前最优动作 | 陷入局部最优 |
| **探索** | 尝试新动作 | 浪费资源 |

In [ ]:
def epsilon_greedy(q_values, epsilon):
    """ε-Greedy 策略"""
    if np.random.random() < epsilon:
        return np.random.randint(len(q_values))
    return np.argmax(q_values)

def softmax_action(q_values, temperature):
    """
    Softmax (Boltzmann) 策略
    
    P(a|s) = exp(Q(s,a)/τ) / Σ exp(Q(s,a')/τ)
    """
    q_scaled = (q_values - np.max(q_values)) / max(temperature, 1e-8)
    exp_q = np.exp(q_scaled)
    probs = exp_q / np.sum(exp_q)
    return np.random.choice(len(q_values), p=probs)

def ucb_action(q_values, action_counts, total_count, c=2.0):
    """
    UCB 策略
    
    A = argmax[Q(s,a) + c * sqrt(ln(t) / N(s,a))]
    """
    if np.any(action_counts == 0):
        return np.random.choice(np.where(action_counts == 0)[0])
    ucb_values = q_values + c * np.sqrt(np.log(total_count + 1) / (action_counts + 1e-8))
    return np.argmax(ucb_values)

In [ ]:
# 演示不同策略
q_values = np.array([1.0, 2.0, 1.5, 0.5])
action_counts = np.array([10, 5, 8, 2])
n_samples = 1000

print(f"Q 值: {q_values}")
print(f"动作计数: {action_counts}\n")

# ε-Greedy
eg_actions = [epsilon_greedy(q_values, 0.1) for _ in range(n_samples)]
eg_dist = [eg_actions.count(i)/n_samples for i in range(4)]
print(f"ε-Greedy (ε=0.1): {[f'{d:.2f}' for d in eg_dist]}")

# Softmax
sm_actions = [softmax_action(q_values, 0.5) for _ in range(n_samples)]
sm_dist = [sm_actions.count(i)/n_samples for i in range(4)]
print(f"Softmax (τ=0.5):  {[f'{d:.2f}' for d in sm_dist]}")

# UCB
ucb_actions = [ucb_action(q_values, action_counts, 100, c=2.0) for _ in range(n_samples)]
ucb_dist = [ucb_actions.count(i)/n_samples for i in range(4)]
print(f"UCB (c=2.0):      {[f'{d:.2f}' for d in ucb_dist]}")

---

## 总结

### 核心要点

1. **Q-Learning** 是无模型、离策略的 TD 控制算法
2. **更新公式**: $Q(S,A) \leftarrow Q(S,A) + \alpha[R + \gamma \max_a Q(S',a) - Q(S,A)]$
3. **离策略特性**: 学习最优策略，与实际行为策略无关
4. **探索策略**: ε-greedy, Softmax, UCB

### 超参数选择

| 参数 | 典型值 | 说明 |
|------|--------|------|
| $\alpha$ | 0.1-0.5 | 学习率 |
| $\gamma$ | 0.99 | 折扣因子 |
| $\epsilon$ | 1.0→0.01 | 探索率 |

### 局限性

- 状态空间必须离散且有限
- 无法处理连续状态
- 无法泛化到未见状态

**下一步**: DQN (深度 Q 网络)

---

## 单元测试

In [ ]:
def run_tests():
    """运行单元测试"""
    print("开始单元测试...\n")
    passed = 0
    
    # 测试1: 环境基本功能
    env = CliffWalkingEnv()
    assert env.reset() == (3, 0), "起始状态错误"
    print("✓ 测试1: 环境初始化")
    passed += 1
    
    # 测试2: 悬崖惩罚
    env.reset()
    _, reward, _ = env.step(1)  # 右移入悬崖
    assert reward == -100.0, "悬崖惩罚错误"
    print("✓ 测试2: 悬崖惩罚")
    passed += 1
    
    # 测试3: Q-Learning 更新
    agent = QLearningAgent(n_actions=4, learning_rate=0.5, discount_factor=0.9)
    state = (0, 0)
    agent.update(state, 0, -1.0, (0, 1), False)
    assert agent.q_table[state][0] == -0.5, "Q值更新错误"
    print("✓ 测试3: Q-Learning 更新")
    passed += 1
    
    # 测试4: 贪心策略
    agent = QLearningAgent(n_actions=4, epsilon=0.0)
    agent.q_table[(0,0)] = np.array([1.0, 2.0, 0.5, 0.5])
    for _ in range(10):
        assert agent.get_action((0,0), training=True) == 1
    print("✓ 测试4: 贪心策略")
    passed += 1
    
    print(f"\n全部 {passed} 项测试通过!")

run_tests()

---

## 参考资料

1. Watkins, C.J.C.H. (1989). Learning from Delayed Rewards. PhD Thesis.
2. Sutton & Barto (2018). Reinforcement Learning: An Introduction, Chapter 6.